In [16]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import joblib
import imodelsx.process_results
import sys
sys.path.append('../experiments/')
results_dir = '../results/07_distill_gmm_models'
experiment_filename = '../experiments/07_distill_gmm_model.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 1466.09it/s]

experiment varied these params: ['dataset_name', 'model_name', 'distiller_name', 'max_depth', 'max_trees']


dataset_name   model_name      distiller_name  max_depth  max_trees
ca_housing     figs            figs            4          20           1
                                                          30           1
                               ft_distill      4          20           1
                                                          30           1
               ft_transformer  figs            4          30           1
                               ft_distill      4          30           1
               resnet          figs            4          30           1
                               ft_distill      4          30           1
               xgboost         figs            5          30           1
                                               6          30           1
                               ft_distill      5          30           1
                                               6          30           1
miami_housing  figs            figs            4        

In [17]:
r_clean = r.drop(columns = ['subsample_frac', 'seed', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique', 
                  'size_interactions', 'gpu', 'n_epochs', 'pre_interaction', 'pre_max_features', 'post_interaction', 
                  'post_max_features', 'max_features'])

In [18]:
r_clean

,dataset_name,model_name,distiller_name,featurizer_name,max_depth,max_rules,max_trees,teacher_r2_score_train_true,teacher_r2_score_val_true,distiller_r2_score_train_true,distiller_r2_score_val_true,distiller_r2_score_train_teacher,distiller_r2_score_val_teacher
0,ca_housing,xgboost,figs,gmm_binary_mapper,5,60,30,0.546980,0.526451,0.528834,0.517085,0.973331,0.965088
1,ca_housing,xgboost,ft_distill,gmm_binary_mapper,5,60,30,0.546980,0.526451,0.520415,0.523593,0.959308,0.956567
2,miami_housing,figs,figs,gmm_binary_mapper,4,60,30,0.861121,0.796868,0.848683,0.793595,0.989835,0.988411
3,miami_housing,xgboost,ft_distill,gmm_binary_mapper,5,60,30,0.940486,0.833889,0.812860,0.776015,0.889742,0.885728
4,ca_housing,figs,ft_distill,gmm_binary_mapper,4,60,30,0.533826,0.524943,0.519649,0.524158,0.973701,0.976963
5,miami_housing,ft_transformer,ft_distill,gmm_binary_mapper,4,60,30,0.580185,0.549817,0.653755,0.619068,0.910555,0.898451
6,miami_housing,figs,ft_distill,gmm_binary_mapper,4,60,20,0.861121,0.796868,0.811433,0.760826,0.926977,0.913161
7,miami_housing,xgboost,figs,gmm_binary_mapper,5,60,30,0.940486,0.833889,0.824014,0.771810,0.902242,0.893327
8,ca_housing,ft_transformer,figs,gmm_binary_mapper,4,60,30,0.486801,0.492607,0.484744,0.498077,0.982933,0.976530
9,miami_housing,ft_transformer,figs,gmm_binary_mapper,4,60,30,0.795064,0.771365,0.748968,0.701420,0.908100,0.874382


In [26]:
r_clean.groupby(['dataset_name', 'model_name', 'distiller_name'])[['teacher_r2_score_val_true', 'distiller_r2_score_val_true', 'distiller_r2_score_val_teacher']].first().round(2)

teacher_r2_score_val_true  \
dataset_name  model_name     distiller_name                              
ca_housing    figs           figs                                 0.52   
                             ft_distill                           0.52   
              ft_transformer figs                                 0.49   
                             ft_distill                           0.49   
              resnet         figs                                 0.47   
                             ft_distill                           0.47   
              xgboost        figs                                 0.53   
                             ft_distill                           0.53   
miami_housing figs           figs                                 0.80   
                             ft_distill                           0.80   
              ft_transformer figs                                 0.77   
                             ft_distill                           0.55   
              resnet         figs                                 0.75   
                             ft_distill                           0.76   
              xgboost        figs                                 0.83   
                             ft_distill                           0.83   

                                             distiller_r2_score_val_true  \
dataset_name  model_name     distiller_name                                
ca_housing    figs           figs                                   0.52   
                             ft_distill                             0.52   
              ft_transformer figs                                   0.50   
                             ft_distill                             0.49   
              resnet         figs                                   0.48   
                             ft_distill                             0.49   
              xgboost        figs                                   0.52   
                             ft_distill                             0.52   
miami_housing figs           figs                                   0.79   
                             ft_distill                             0.76   
              ft_transformer figs                                   0.70   
                             ft_distill                             0.62   
              resnet         figs                                   0.71   
                             ft_distill                             0.76   
              xgboost        figs                                   0.77   
                             ft_distill                             0.78   

                                             distiller_r2_score_val_teacher  
dataset_name  model_name     distiller_name                                  
ca_housing    figs           figs                                      1.00  
                             ft_distill                                0.98  
              ft_transformer figs                                      0.98  
                             ft_distill                                0.96  
              resnet         figs                                      0.96  
                             ft_distill                                0.93  
              xgboost        figs                                      0.97  
                             ft_distill                                0.96  
miami_housing figs           figs                                      0.99  
                             ft_distill                                0.91  
              ft_transformer figs                                      0.87  
                             ft_distill                                0.90  
              resnet         figs                                      0.88  
                             ft_distill                                0.93  
              xgboost        figs                                      0.89  
                 

In [ ]:
r.groupby(['dataset_name', 'model_name'])[['teacher_r2_score_val_true']].mean()

In [ ]:
os.getpid()

In [ ]:
datasets = r['dataset_name'].unique()

In [ ]:
#RF params: max_depth, max_features
random_forest= r[r['model_name'] == 'random_forest'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions'])
random_forest_g = random_forest.groupby(['dataset_name','model_name', 'max_depth', 'max_features', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()

In [ ]:
random_forest_g

In [ ]:
# random_forest_best_hyp = {}
# for d in datasets:
#     queried = random_forest_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name', 'max_depth', 'max_features'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     random_forest_best_hyp[d] = d_best_hyp
# random_forest_best_hyp

In [ ]:
#RF+ params: max_depth, max_features
rf_plus = r[r['model_name'] == 'rf_plus'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions'])
rf_plus_g = rf_plus.groupby(['dataset_name','model_name', 'max_depth', 'max_features', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
rf_plus_g

In [ ]:
# rf_plus_best_hyp = {}
# for d in datasets:
#     queried = rf_plus_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name', 'max_depth', 'max_features'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     rf_plus_best_hyp[d] = d_best_hyp
# rf_plus_best_hyp

In [ ]:
#FIGS params: max_rules, max_trees, max_features
figs = r[r['model_name'] == 'figs'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_depth','pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions'])
figs_g = figs.groupby(['dataset_name','model_name', 'max_rules','max_trees', 'max_features', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
figs_g

In [ ]:
#RF+ params: max_depth, max_features
xgboost = r[r['model_name'] == 'xgboost'].drop(columns=['subsample_frac', 'save_dir','use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'n_epochs', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions', 'max_features'])
xgboost_g = xgboost.groupby(['dataset_name','model_name', 'max_depth', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
xgboost_g

In [ ]:
# xgboost_best_hyp = {}
# for d in datasets:
#     queried = xgboost_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name', 'max_depth'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     xgboost_best_hyp[d] = d_best_hyp
# xgboost_best_hyp

In [ ]:
#ResNet params: n_epochs
resnet = r[r['model_name'] == 'resnet'].drop(columns=['subsample_frac', 'save_dir','use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions', 'max_features', 'max_depth'])
resnet_g = resnet.groupby(['dataset_name','model_name', 'n_epochs', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
resnet_g

In [ ]:
# resnet_best_hyp = {}
# for d in resnet['dataset_name'].unique():
#     queried = resnet_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name','n_epochs'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     resnet_best_hyp[d] = d_best_hyp
# resnet_best_hyp

In [ ]:
#FT Transformer params: n_epochs
ft_transformer = r[r['model_name'] == 'ft_transformer'].drop(columns=['subsample_frac', 'save_dir', 'use_cache', 'cat_mappings', 'task_type', 'save_dir_unique',
                                                        'gpu', 'max_rules', 'max_trees', 'pre_interaction', 'post_interaction',
                                                        'pre_max_features', 'post_max_features', 'size_interactions', 'max_features', 'max_depth'])
ft_transformer_g = ft_transformer.groupby(['dataset_name','model_name', 'n_epochs', 'bit', 'depth'])[['teacher_r2_score_train_true', 'teacher_r2_score_val_true']].mean()
ft_transformer_g

In [ ]:
# ft_transformer_best_hyp = {}
# for d in ft_transformer['dataset_name'].unique():
#     queried = ft_transformer_g.query(f"dataset_name == '{d}'")
#     d_best_hyp = dict(zip(['dataset_name','model_name','n_epochs'], list(queried['r2_score_val_true'].idxmax())))
#     d_best_hyp['r2_val'] = queried['r2_score_val_true'].max()
#     del d_best_hyp['dataset_name']
#     del d_best_hyp['model_name']
#     ft_transformer_best_hyp[d] = d_best_hyp
# ft_transformer_best_hyp

In [ ]:
best_hyp = {}
for m in ['random_forest', 'rf_plus', 'figs', 'xgboost', 'resnet', 'ft_transformer']:
    best_hyp[m] = eval(f'{m}_best_hyp')

In [ ]:
import json 

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open('/home/mattyshen/interpretableDistillation/scripts/best_hyperparams/original_hyperparams.json', "w") as outfile: 
    json.dump(best_hyp, outfile, cls =NpEncoder)